In [41]:
#Importing Statements
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [3]:
# Importing Dataset
news= pd.read_csv("Dataset/train.csv")
news.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
# Null Values
news.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
# Dropping Null Values and Resetting Index
news.dropna(inplace=True)
news.reset_index(inplace=True)

In [6]:
# Extracting Independent and Dependent Features
X=news.drop('label',axis=1)
y=news['label']

Data Preprocessing

In [7]:
text=X.copy()

In [10]:
ps= PorterStemmer()
corpus=[]
for row in range (0, len(text)):
    processed_text= re.sub('[^a-zA-Z]', ' ', text['title'][row])
    processed_text=processed_text.lower()
    processed_text=processed_text.split()
    processed_text= [ps.stem(word) for word in processed_text if not word in stopwords.words('english')]
    processed_text=' '.join(processed_text)
    corpus.append(processed_text)

In [11]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

One Hot Representation

In [12]:
voc_size=5000

In [13]:
one_hot_repr= [one_hot(words, voc_size) for words in corpus]
one_hot_repr[0]

[1983, 2721, 3232, 3293, 2056, 3172, 4415, 3299, 3913, 3145]

Embedding

In [14]:
sent_length= 20
embedded_text= pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)
embedded_text[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1983,
       2721, 3232, 3293, 2056, 3172, 4415, 3299, 3913, 3145])

Model

In [16]:
features=40
model=Sequential()
model.add(Embedding(voc_size,features,input_length= sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


Train Test Split

In [21]:
X= np.array(embedded_text)
y=np.array(y)
print(X.shape, y.shape)

(18285, 20) (18285,)


In [23]:
X_train, X_test,y_train,y_test= train_test_split(X,y, test_size=0.33,random_state=123)

Model Training

In [24]:
model.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 21s 52ms/step - loss: 0.3331 - accuracy: 0.8420 - val_loss: 0.1988 - val_accuracy: 0.9183
Epoch 2/10
192/192 [==============================] - 11s 56ms/step - loss: 0.1409 - accuracy: 0.9454 - val_loss: 0.1961 - val_accuracy: 0.9218
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0998 - accuracy: 0.9638 - val_loss: 0.2085 - val_accuracy: 0.9163
Epoch 4/10
192/192 [==============================] - 8s 41ms/step - loss: 0.0776 - accuracy: 0.9718 - val_loss: 0.2645 - val_accuracy: 0.9188
Epoch 5/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0574 - accuracy: 0.9808 - val_loss: 0.2490 - val_accuracy: 0.9148
Epoch 6/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0431 - accuracy: 0.9870 - val_loss: 0.3278 - val_accuracy: 0.9130
Epoch 7/10
192/192 [==============================] - 8s 41ms/step - loss: 0.0310 - accuracy: 0.9913 - val_loss: 0.3964 - val_accuracy: 0.91

Model2 (Using Droput)

In [28]:
features=40
model2=Sequential()
model2.add(Embedding(voc_size,features,input_length= sent_length))
model2.add(Dropout(0.3))
model2.add(LSTM(100))
model2.add(Dropout(0.3))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model2.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_2 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
model2.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 17s 51ms/step - loss: 0.3527 - accuracy: 0.8274 - val_loss: 0.2063 - val_accuracy: 0.9148
Epoch 2/10
192/192 [==============================] - 9s 47ms/step - loss: 0.1508 - accuracy: 0.9401 - val_loss: 0.1969 - val_accuracy: 0.9218
Epoch 3/10
192/192 [==============================] - 9s 46ms/step - loss: 0.1075 - accuracy: 0.9602 - val_loss: 0.2013 - val_accuracy: 0.9211
Epoch 4/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0822 - accuracy: 0.9709 - val_loss: 0.2281 - val_accuracy: 0.9135
Epoch 5/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0631 - accuracy: 0.9779 - val_loss: 0.2595 - val_accuracy: 0.9147
Epoch 6/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0487 - accuracy: 0.9847 - val_loss: 0.2773 - val_accuracy: 0.9125
Epoch 7/10
192/192 [==============================] - 8s 44ms/step - loss: 0.0361 - accuracy: 0.9879 - val_loss: 0.3346 - val_accuracy: 0.909

Performance Metrics 

In [46]:
y_pred=model2.predict(X_test)
class_y= np.argmax(y_pred,axis=1)

189/189 [==============================] - 4s 19ms/step


In [47]:
confusion_matrix(y_test,class_y)

array([[3406,    0],
       [2629,    0]], dtype=int64)

In [48]:
accuracy_score(y_test,class_y)

0.5643744821872411